# Lambda School Data Science Module 141
## Statistics, Probability, and Inference

## Prepare - examine what's available in SciPy

As we delve into statistics, we'll be using more libraries - in particular the [stats package from SciPy](https://docs.scipy.org/doc/scipy/reference/tutorial/stats.html).

In [58]:
from scipy import stats
dir(stats)

['__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_binned_statistic',
 '_constants',
 '_continuous_distns',
 '_discrete_distns',
 '_distn_infrastructure',
 '_distr_params',
 '_multivariate',
 '_stats',
 '_stats_mstats_common',
 '_tukeylambda_stats',
 'absolute_import',
 'alpha',
 'anderson',
 'anderson_ksamp',
 'anglit',
 'ansari',
 'arcsine',
 'argus',
 'bartlett',
 'bayes_mvs',
 'bernoulli',
 'beta',
 'betaprime',
 'binned_statistic',
 'binned_statistic_2d',
 'binned_statistic_dd',
 'binom',
 'binom_test',
 'boltzmann',
 'boxcox',
 'boxcox_llf',
 'boxcox_normmax',
 'boxcox_normplot',
 'bradford',
 'burr',
 'burr12',
 'cauchy',
 'chi',
 'chi2',
 'chi2_contingency',
 'chisquare',
 'circmean',
 'circstd',
 'circvar',
 'combine_pvalues',
 'contingency',
 'cosine',
 'crystalball',
 'cumfreq',
 'describe',
 'dgamma',
 'dirichlet',
 'distributions',
 'division',
 'dlaplace',
 'dweibull',
 'energy_dist

In [59]:
# As usual, lots of stuff here! There's our friend, the normal distribution
norm = stats.norm()
print(norm.mean())
print(norm.std())
print(norm.var())

0.0
1.0
1.0


In [60]:
# And a new friend - t
t1 = stats.t(5)  # 5 is df "shape" parameter
print(t1.mean())
print(t1.std())
print(t1.var())

0.0
1.2909944487358056
1.6666666666666667


![T distribution PDF with different shape parameters](https://upload.wikimedia.org/wikipedia/commons/4/41/Student_t_pdf.svg)

*(Picture from [Wikipedia](https://en.wikipedia.org/wiki/Student's_t-distribution#/media/File:Student_t_pdf.svg))*

The t-distribution is "normal-ish" - the larger the parameter (which reflects its degrees of freedom - more input data/features will increase it), the closer to true normal.

In [61]:
t2 = stats.t(30)  # Will be closer to normal
print(t2.mean())
print(t2.std())
print(t2.var())

0.0
1.0350983390135313
1.0714285714285714


Why is it different from normal? To better reflect the tendencies of small data and situations with unknown population standard deviation. In other words, the normal distribution is still the nice pure ideal in the limit (thanks to the central limit theorem), but the t-distribution is much more useful in many real-world situations.

History sidenote - this is "Student":

![William Sealy Gosset](https://upload.wikimedia.org/wikipedia/commons/4/42/William_Sealy_Gosset.jpg)

*(Picture from [Wikipedia](https://en.wikipedia.org/wiki/File:William_Sealy_Gosset.jpg))*

His real name is William Sealy Gosset, and he published under the pen name "Student" because he was not an academic. He was a brewer, working at Guinness and using trial and error to determine the best ways to yield barley. He's also proof that, even 100 years ago, you don't need official credentials to do real data science!

## Live Lecture - let's perform and interpret a t-test

We'll generate our own data, so we can know and alter the "ground truth" that the t-test should find. We will learn about p-values and how to interpret "statistical significance" based on the output of a hypothesis test.

In [62]:
# TODO - during class, but please help!

## Assignment - apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [63]:
!pip install pandas
!pip install scipy

In [64]:
import pandas as pd
from scipy import stats

In [65]:
house_votes = pd.read_csv(
    'https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data',
    na_values = '?',
    names = [ 'party', 
            'handicapped-infants', 'water-project-cost-sharing', 
           'adoption-of-the-budget-resolution', 'physician-fee-freeze', 
           'el-salvador-aid', 'religious-groups-in-schools', 
           'anti-satellite-test-ban', 'aid-to-nicaraguan-contras', 
           'mx-missile', 'immigration', 'synfuels-corporation-cutback', 
           'education-spending', 'superfund-right-to-sue', 'crime', 
           'duty-free-exports', 'export-administration-act-south-africa'])
house_votes.head()

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


In [66]:
house_votes.describe()

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
count,435,423,387,424,424,420,424,421,420,413,428,414,404,410,418,407,331
unique,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
top,democrat,n,y,y,n,y,y,y,y,y,y,n,n,y,y,n,y
freq,267,236,195,253,247,212,272,239,242,207,216,264,233,209,248,233,269


In [67]:
house_votes.isnull().sum().sum()

392

In [68]:
house_votes.shape

(435, 17)

In [69]:
house_votes.isnull().sum()

'''
  Since the number of nulls per issue is not much compared
  to the number of representatives (435), we can drop
  the null values without affecting the ttest.
'''

'\n  Since the number of nulls per issue is not much compared\n  to the number of representatives (435), we can drop\n  the null values without affecting the ttest.\n'

In [70]:
# house_votes.replace(to_replace = {'y': 1, 'n': 0}, inplace = True)
changed = house_votes.replace(to_replace = {'y': int(1), 'n': int(0)})
changed.head()

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [71]:
for issue in changed.columns:
    if issue == 'party':
        continue
        
    # Drop null values since there are not that many
    # compared to the number of representatives (435).
    df = changed[['party', issue]].dropna()
    rep = df[df.party == 'republican']
    dem = df[df.party == 'democrat']
    print('issue:', issue, '\n  ',
          scipy.stats.ttest_ind(rep[issue], dem[issue]),
         '\n')

issue: handicapped-infants 
   Ttest_indResult(statistic=-9.205264294809222, pvalue=1.613440327936998e-18) 

issue: water-project-cost-sharing 
   Ttest_indResult(statistic=0.08896538137868286, pvalue=0.9291556823994811) 

issue: adoption-of-the-budget-resolution 
   Ttest_indResult(statistic=-23.21277691701378, pvalue=2.0703402795405602e-77) 

issue: physician-fee-freeze 
   Ttest_indResult(statistic=49.367081573014055, pvalue=1.994262314074572e-177) 

issue: el-salvador-aid 
   Ttest_indResult(statistic=21.136692611732194, pvalue=5.600520111728605e-68) 

issue: religious-groups-in-schools 
   Ttest_indResult(statistic=9.737575825219457, pvalue=2.393672252059893e-20) 

issue: anti-satellite-test-ban 
   Ttest_indResult(statistic=-12.526187929077842, pvalue=8.521033017443427e-31) 

issue: aid-to-nicaraguan-contras 
   Ttest_indResult(statistic=-18.052093200819733, pvalue=2.824718413723432e-54) 

issue: mx-missile 
   Ttest_indResult(statistic=-16.43750326854299, pvalue=5.03079265310788

  ## Conclusion:
  
  * Democrats favor 'Adoption of the budget resolution' over Republicans
  The above printout shows:

    issue: adoption-of-the-budget-resolution 
      Ttest_indResult(statistic=-23.21277691701378, pvalue=2.0703402795405602e-77)
      
  Since we're passing Republicans as the first array into ttest_int(),
  this tells us the democrats were in favor of the 'adoption-of-the-budget-resolution'
  issue.  The p-value is almost 0.
  
  
  * Republicans favor 'Physician fee freeze' over Democrats
    issue: physician-fee-freeze 
      Ttest_indResult(statistic=49.367081573014055, pvalue=1.994262314074572e-177) 
  The physician-fee-freeze is supported by the Republicans, more than the Democrats.
  We can say the p-value is 0.  This p-value revokes the null-hypothesis, and
  is supports the alternate hypothesis of Republicans favoring fee freezes for
  physicians.
  
  
  * Republicans and Democrats equally support 'Water Project Cost Sharing'
    issue: water-project-cost-sharing 
      Ttest_indResult(statistic=0.08896538137868286, pvalue=0.9291556823994811)
  The water-project-cost-sharing issue seems to have slightly larger Republican
  support (based on the t-statistic). The p-value is so high that it supports
  the null hypothesis.  So in this case, both Republicans and Democrats
  support the water-project-cost-sharing equally.